### Eksport modelu ensemble do formatu ONNX

In [1]:
# Komórka 1: Importy i konfiguracja początkowa
import glob
import os
import torch
from datetime import datetime
import numpy as np
import librosa

# Importy z Twojego projektu
from export_scripts.export_onnx_notebook_helper import export_model_to_onnx
from export_scripts.callibration_utils import EnsembleCalibrationDataReader
from config import (
    BATCH_SIZE, DEVICE, TEST_SPLIT, CLASS_NAMES, FEATURE_RESULTS_DIR, 
    PROCESSED_FEATURES_DIR, ENSEMBLE_OUTPUT_DIR, OPTUNA_TRIALS,
    MAX_LENGTH, N_FFT, HOP_LENGTH, N_MELS, N_MFCC, N_CHROMA
)

# Funkcja do automatycznego konfigurowania ensemble modelu
def auto_configure_ensemble(models_base_dir=None, ensemble_save_dir=None, best_models_only=True):
    """
    Automatycznie konfiguruje model ensemble na podstawie dostępnych modeli.
    
    Args:
        models_base_dir: Ścieżka bazowa do katalogu z modelami
        ensemble_save_dir: Ścieżka do zapisu modelu ensemble
        best_models_only: Czy używać tylko najlepszych modeli
        
    Returns:
        tuple: (model_ensemble, feature_types)
    """
    from helpers.ensemble_model import WeightedEnsembleModel
    from helpers.utils import load_pretrained_model
    from helpers.resnet_model_definition import AudioResNet
    
    # Jeśli nie podano ścieżek, użyj domyślnych
    if models_base_dir is None:
        models_base_dir = FEATURE_RESULTS_DIR
    
    if ensemble_save_dir is None:
        ensemble_save_dir = ENSEMBLE_OUTPUT_DIR
    
    # Szukaj modeli dla różnych typów cech
    feature_types = ['melspectrogram', 'mfcc', 'chroma']
    model_paths = {}
    models_dict = {}
    
    for feature_type in feature_types:
        # Wzorzec dla najlepszych modeli
        if best_models_only:
            pattern = f"{models_base_dir}/{feature_type}/best_model_{feature_type}_*.pt"
        else:
            pattern = f"{models_base_dir}/{feature_type}/model_{feature_type}_*.pt"
        
        # Szukaj modeli pasujących do wzorca
        model_files = glob.glob(pattern)
        
        if model_files:
            # Wybierz najnowszy model (zakładając, że nazwy plików zawierają timestamp)
            newest_model = sorted(model_files)[-1]
            model_paths[feature_type] = newest_model
            
            # Załaduj model
            print(f"Ładowanie modelu dla {feature_type}: {os.path.basename(newest_model)}")
            model = load_pretrained_model(
                model_path=newest_model,
                model_class=AudioResNet,
                num_classes=len(CLASS_NAMES),
                device='cpu'  # Używanie CPU do eksportu
            )
            
            if model is not None:
                models_dict[feature_type] = model
            else:
                print(f"Ostrzeżenie: Nie udało się załadować modelu dla {feature_type}")
    
    # Sprawdź, czy znaleziono jakiekolwiek modele
    if not models_dict:
        raise ValueError("Nie znaleziono żadnych modeli! Sprawdź ścieżki do katalogów z modelami.")
    
    # Utwórz model ensemble
    ensemble_model = WeightedEnsembleModel(models_dict, weights=None)
    
    return ensemble_model, list(models_dict.keys())

# Komórka 2: Załadowanie lub utworzenie modelu ensemble
# Sprawdź, czy model ensemble jest już dostępny w sesji
ensemble_model_loaded = None
if 'ensemble_model' in locals() or 'ensemble_model' in globals():
    print("Model ensemble już załadowany w sesji.")
    ensemble_model_loaded = ensemble_model
else:
    # Spróbuj znaleźć zapisany model ensemble
    ensemble_model_path = None
    ensemble_patterns = [
        "ensemble_outputs/*/models/best_ensemble_model.pt",  # Najlepsze modele z Optuna
        "ensemble_outputs/*/models/ensemble_model.pt",       # Modele bez optymalizacji
        "exported_models/*/best_ensemble_model.pt",          # Eksportowane najlepsze modele
        "exported_models/*/ensemble_model.pt"                # Eksportowane modele
    ]
    
    for pattern in ensemble_patterns:
        model_files = glob.glob(pattern)
        if model_files:
            # Sortuj według daty modyfikacji - najnowszy na końcu
            ensemble_model_path = sorted(model_files, key=os.path.getmtime)[-1]
            print(f"Znaleziono model ensemble: {ensemble_model_path}")
            break
    
    # Jeśli znaleziono model, spróbuj go załadować
    if ensemble_model_path:
        try:
            from helpers.ensemble_model import WeightedEnsembleModel
            from helpers.resnet_model_definition import AudioResNet
            
            # Załaduj modele bazowe (potrzebne do rekonstrukcji)
            _, feature_types = auto_configure_ensemble()
            
            # Utwórz słownik modeli bazowych
            from helpers.utils import load_pretrained_model
            models_dict = {}
            for ft in feature_types:
                model = AudioResNet(num_classes=len(CLASS_NAMES))
                models_dict[ft] = model
            
            # Załaduj model ensemble
            ensemble_model_loaded, _ = WeightedEnsembleModel.load(ensemble_model_path, models_dict)
            print(f"Model ensemble załadowany z {ensemble_model_path}")
            print(f"Typy cech: {ensemble_model_loaded.feature_types}")
            print(f"Wagi: {ensemble_model_loaded.get_weights()}")
        except Exception as e:
            print(f"Błąd podczas ładowania modelu ensemble: {e}")
            ensemble_model_loaded = None
    
    # Jeśli nie udało się załadować modelu, utwórz nowy
    if ensemble_model_loaded is None:
        try:
            print("Tworzenie nowego modelu ensemble...")
            ensemble_model_loaded, feature_types = auto_configure_ensemble()
            print(f"Utworzono model ensemble z typami cech: {feature_types}")
        except Exception as e:
            print(f"Błąd podczas tworzenia modelu ensemble: {e}")
            raise

# Przypisz załadowany model do zmiennej ensemble_model dla dalszego użycia
ensemble_model = ensemble_model_loaded
print(f"Model ensemble gotowy do eksportu. Typy cech: {ensemble_model.feature_types}")

# Komórka 3: Eksport modelu do ONNX
# Utwórz katalog wyjściowy z timestampem
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
output_dir = f"exported_models/onnx_{timestamp}"
os.makedirs(output_dir, exist_ok=True)
print(f"Katalog wyjściowy: {output_dir}")

# Konfiguracja eksportu i kwantyzacji
quantize_model = True         # Czy kwantyzować model po eksporcie
quantization_type = "static"  # Typ kwantyzacji (static/dynamic) - dla CNN zalecana static
activation_type = "int8"      # Typ danych aktywacji (int8 dla modeli CNN)
weight_type = "int8"          # Typ danych wag (int8 dla modeli CNN)
quant_format = "qdq"          # Format kwantyzacji (qdq jest zalecane dla modeli CNN)
per_channel = True            # Kwantyzacja per-channel dla lepszej dokładności

# Przygotowanie danych kalibracyjnych dla kwantyzacji statycznej
print("\nPrzygotowywanie danych kalibracyjnych dla kwantyzacji statycznej...")

# Załadowanie próbek audio do kalibracji
audio_samples = []
num_samples = 10  # Liczba próbek audio do kalibracji

# Szukaj plików audio w różnych katalogach
audio_paths = []
data_dirs = ['data/test', 'data/valid', 'data_processed/test', 'data_processed/valid']

for data_dir in data_dirs:
    if os.path.exists(data_dir):
        for ext in ['.wav', '.mp3', '.ogg']:
            audio_files = glob.glob(f"{data_dir}/**/*{ext}", recursive=True)
            if audio_files:
                audio_paths.extend(audio_files[:num_samples])
                if len(audio_paths) >= num_samples:
                    break
        if len(audio_paths) >= num_samples:
            break

# Wczytaj pliki audio lub wygeneruj sztuczne dane, jeśli nie znaleziono
if not audio_paths:
    print("Nie znaleziono plików audio. Generowanie sztucznych danych...")
    
    # Generowanie sztucznych danych
    for i in range(num_samples):
        t = np.linspace(0, MAX_LENGTH, int(22050 * MAX_LENGTH), endpoint=False)
        audio = np.sin(2 * np.pi * 440 * t) * 0.3  # Ton A4 (440 Hz)
        audio += np.sin(2 * np.pi * 880 * t) * 0.2  # Harmoniczna
        audio += np.random.normal(0, 0.1, size=len(t))  # Szum
        audio = audio / np.max(np.abs(audio))  # Normalizacja
        audio_samples.append(audio)
        print(f"Wygenerowano sztuczną próbkę audio {i+1}/{num_samples}")
else:
    # Ładowanie rzeczywistych plików audio
    for i, path in enumerate(audio_paths[:num_samples]):
        try:
            audio, sr = librosa.load(path, sr=22050)
            target_length = int(MAX_LENGTH * sr)
            if len(audio) > target_length:
                audio = audio[:target_length]
            else:
                audio = np.pad(audio, (0, max(0, target_length - len(audio))), mode='constant')
            audio_samples.append(audio)
            print(f"Załadowano próbkę {i+1}/{len(audio_paths[:num_samples])}: {os.path.basename(path)}")
        except Exception as e:
            print(f"Błąd ładowania {path}: {e}")

# Utwórz czytnik danych kalibracyjnych
feature_types = ensemble_model.feature_types  # Pobierz typy cech z modelu
print(f"Typy cech modelu: {feature_types}")

calibration_reader = EnsembleCalibrationDataReader(
    audio_samples=audio_samples,
    feature_types=feature_types,
    sample_rate=22050
)

print(f"Utworzono czytnik danych kalibracyjnych z {len(audio_samples)} próbkami audio")

print("\nRozpoczynam eksport modelu...")

# Eksport modelu do formatu ONNX
try:
    export_result = export_model_to_onnx(
        ensemble_model=ensemble_model,
        output_dir=output_dir,
        class_names=CLASS_NAMES,
        export_params={
            "quantize_model": quantize_model,
            "quantization_type": quantization_type,
            "activation_type": activation_type,
            "weight_type": weight_type,
            "quant_format": quant_format,
            "per_channel": per_channel,
            "reduce_range": False,  # True tylko dla starszych CPU bez wspierania instrukcji VNNI
            "use_cuda": torch.cuda.is_available(),
            "opset_version": 17  # Używaj najnowszej wersji opset
        },
        calibration_data_reader=calibration_reader  # Przekazanie czytnika danych kalibracyjnych
    )
except Exception as e:
    print(f"Błąd podczas eksportu modelu: {e}")
    import traceback
    traceback.print_exc()
    raise

if export_result.get("success", False):
    print("\nEksport modelu ensemble do ONNX zakończony sukcesem!")
    print(f"Model ONNX zapisany w: {export_result['onnx_path']}")
    print(f"Zoptymalizowany model ONNX zapisany w: {export_result['optimized_onnx_path']}")
    
    # Rozmiar modelu
    original_size_mb = export_result['sizes_mb'].get('original', 0)
    optimized_size_mb = export_result['sizes_mb'].get('optimized', 0)
    print(f"Rozmiar oryginalnego modelu: {original_size_mb:.2f} MB")
    print(f"Rozmiar zoptymalizowanego modelu: {optimized_size_mb:.2f} MB")
    print(f"Redukcja rozmiaru po optymalizacji: {export_result.get('optimization_reduction', 0):.2f}%")
    
    # Sprawdź czy kwantyzacja została wykonana
    if "quantized_onnx_path" in export_result and export_result['quantized_onnx_path'] and os.path.exists(export_result['quantized_onnx_path']):
        quantized_size_mb = export_result['sizes_mb'].get('quantized', 0)
        print(f"Skwantyzowany model ONNX zapisany w: {export_result['quantized_onnx_path']}")
        print(f"Rozmiar skwantyzowanego modelu: {quantized_size_mb:.2f} MB")
        print(f"Redukcja rozmiaru po kwantyzacji: {export_result.get('quantization_reduction_vs_source', 0):.2f}%")
        
        # Jeśli użyto kwantyzacji statycznej, pokaż szczegóły
        if export_result.get("quantization_type_used") == "static":
            print(f"\nSzczegóły kwantyzacji statycznej:")
            if "quantization_params_used" in export_result:
                params = export_result["quantization_params_used"] 
                print(f"- Używana parametryzacja: {params}")
                if "op_types_to_quantize" in params and params["op_types_to_quantize"]:
                    print(f"- Kwantyzowane operacje: {params['op_types_to_quantize']}")
    else:
        print("Kwantyzacja nie została przeprowadzona lub nie powiodła się.")
        if "quantization_error" in export_result:
            print(f"Błąd kwantyzacji: {export_result['quantization_error']}")
    
    # Informacje o metadanych i weryfikacji
    print(f"Metadane modelu zapisane w: {export_result.get('metadata_path', 'N/A')}")
    
    # Wyświetl wyniki weryfikacji
    if "verification_result" in export_result:
        result = export_result["verification_result"]
        if result.get("success", False):
            print("\nWeryfikacja modelu zakończona powodzeniem")
            if "max_diff" in result:
                print(f"Maksymalna różnica między PyTorch i ONNX: {result['max_diff']:.6f}")
            if "has_warning" in result and result["has_warning"]:
                print(f"Ostrzeżenie: {result.get('warning', 'Wykryto różnice, ale mieszczą się w tolerancji')}")
        else:
            print(f"\nWeryfikacja modelu nie powiodła się: {result.get('error', 'Nieznany błąd')}")
else:
    print("\nEksport modelu się nie powiódł")
    print(f"Błąd: {export_result.get('error', 'Nieznany błąd')}") 

Znaleziono model ensemble: ensemble_outputs\ensemble_run_20250519_150653\models\best_ensemble_model.pt
Ładowanie modelu dla melspectrogram: best_model_melspectrogram_20250517_001727.pt
Model załadowany pomyślnie z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/melspectrogram\best_model_melspectrogram_20250517_001727.pt
Ładowanie modelu dla mfcc: best_model_mfcc_20250517_004119.pt
Model załadowany pomyślnie z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/mfcc\best_model_mfcc_20250517_004119.pt
Ładowanie modelu dla chroma: best_model_chroma_20250517_005304.pt
Model załadowany pomyślnie z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/chroma\best_model_chroma_20250517_005304.pt
Załadowano model z weights_only=True
Model ensemble załadowany z ensemble_outputs\ensemble_run_20250519_150653\models\best_ensemble_model.pt
Typy cech: ['melspect

Shape inference (static quant) zakończone: C:\Users\kubas\AppData\Local\Temp\onnx_quant_static_fvpk7ww9\inferred_shape_model_static.onnx.

Próba kwantyzacji statycznej (1/3): Podstawowa statyczna int8/int8 qdq dla CNN
  Próba kwantyzacji statycznej (Podstawowa statyczna int8/int8 qdq dla CNN) nie powiodła się: Only an existing tensor can be modified, '/ensemble/Softmax_output_0' is not.

Próba kwantyzacji statycznej (2/3): Statyczna z wykluczeniem operacji pomocniczych
  Wykluczanie węzłów (wg nazw): ['/ensemble/Softmax', '/ensemble/Softmax_1', '/ensemble/Softmax_2'] dla typów ['Softmax', 'ReduceMean', 'Tanh', 'Sigmoid']


  Próba kwantyzacji statycznej (Statyczna z wykluczeniem operacji pomocniczych) nie powiodła się: No data is collected.

Próba kwantyzacji statycznej (3/3): Statyczna uproszczona
  Wykluczanie węzłów (wg nazw): ['/ensemble/Softmax', '/ensemble/Softmax_1', '/ensemble/Softmax_2'] dla typów ['Softmax', 'ReduceMean', 'Tanh', 'Sigmoid']


  Próba kwantyzacji statycznej (Statyczna uproszczona) nie powiodła się: No data is collected.
Wszystkie próby kwantyzacji statycznej nieudane.
Ostrzeżenie: Kwantyzacja static nie powiodła się: Wszystkie próby kwantyzacji statycznej nieudane. Ostatni błąd: No data is collected.

Model wybrany do weryfikacji: NIESKWANTYZOWANY (exported_models/onnx_20250520_174735\ensemble_model.onnx)

--- Krok 4: Weryfikacja finalnego modelu ONNX ---
Dostępni dostawcy ONNX Runtime: ['AzureExecutionProvider', 'CPUExecutionProvider']
Używam CPU do inferencji ONNX (wersja ONNX Runtime: 1.19.2)
Używam 8 wątków CPU do inferencji
Utworzono sesję ONNX Runtime dla modelu: exported_models/onnx_20250520_174735\ensemble_model.onnx
Użyte dostawce (providers): ['CPUExecutionProvider']
Oczekiwane wejścia:
  - mel_input: ['batch_size', 1, 128, 'time_steps'] (tensor(float))
  - mfcc_input: ['batch_size', 1, 40, 'time_steps'] (tensor(float))
  - chroma_input: ['batch_size', 1, 12, 'time_steps'] (tensor(float))
Wyjścia m

In [10]:
# Import funkcji eksportu
import glob
import os
import torch
import inspect
from datetime import datetime
from export_scripts.export_onnx_notebook_helper import export_model_to_onnx
from config import BATCH_SIZE, DEVICE,TEST_SPLIT, CLASS_NAMES,FEATURE_RESULTS_DIR, PROCESSED_FEATURES_DIR, ENSEMBLE_OUTPUT_DIR, OPTUNA_TRIALS, OPTUNA_TIMEOUT, CV_FOLDS, SEED
from helpers.resnet_model_definition import AudioResNet
from helpers.ensemble_trainer import EnsembleModelTrainer



print(f"Używane urządzenie: {DEVICE}")
print(f"Wersja PyTorch: {torch.__version__}")

Używane urządzenie: cpu
Wersja PyTorch: 2.7.0+cpu


In [11]:
def find_latest_model(feature_type):
    """Zwraca najnowszy model dla danej reprezentacji audio."""
    search_pattern = f"{FEATURE_RESULTS_DIR}/{feature_type}/best_model_{feature_type}_*.pt"
    model_files = glob.glob(search_pattern)
    
    if not model_files:
        return None
    
    # Sortowanie plików według daty w nazwie (najnowszy na końcu)
    model_files.sort(key=lambda x: os.path.basename(x).split('_')[-1].split('.')[0])
    return model_files[-1]

def find_feature_file(feature_type):
    """Zwraca plik z cechami dla danej reprezentacji audio."""
    search_pattern = f"{PROCESSED_FEATURES_DIR}/{feature_type}_*.pkl"
    feature_files = glob.glob(search_pattern)
    
    if not feature_files:
        return None
    
    # W przypadku wielu plików, wybór najnowszego
    if len(feature_files) > 1:
        feature_files.sort(key=os.path.getmtime, reverse=True)
    
    return feature_files[0]

def auto_configure_ensemble(feature_types=None):
    """
    Konfiguracja modelu ensemble w sposób automatyczny.
    """
    # Wykorzystanie zmiennych z config.py
    if feature_types is None:
        try:
            feature_types = [d for d in os.listdir(FEATURE_RESULTS_DIR) 
                             if os.path.isdir(os.path.join(FEATURE_RESULTS_DIR, d))]
        except FileNotFoundError:
             raise FileNotFoundError(f"Nie znaleziono katalogu '{FEATURE_RESULTS_DIR}'.")
    
    print(f"Typy cech wybrane do modelu ensemble: {', '.join(feature_types)}")

    model_paths = {}
    feature_files = {}
    initial_weights = {}
    missing_feature_files = []
    
    for feature_type in feature_types:
        # Weryfikacja pliku cech
        feature_file = find_feature_file(feature_type)
        if not feature_file:
            missing_feature_files.append(feature_type)
            print(f"Ostrzeżenie: Brak pliku cech dla {feature_type}")
            continue # Kontynuacja do następnego typu, bez szukania modelu, gdy brak cech
        
        feature_files[feature_type] = feature_file
        
        # Wyszukiwanie modelu
        model_path = find_latest_model(feature_type)
        if model_path:
            model_paths[feature_type] = model_path
            initial_weights[feature_type] = 1.0 # Waga do normalizacji w późniejszym etapie
        else:
            print(f"Ostrzeżenie: Brak modelu dla {feature_type}, mimo istnienia pliku cech.")
            # Decyzja o kontynuacji bez tego modelu lub zgłoszenie błędu.
            # Na razie pominięcie tego typu cechy w modelu.
            del feature_files[feature_type] # Usunięcie wpisu o pliku cech, gdy brak modelu
            
    # W przypadku brakujących plików cech, zgłoszenie błędu
    if missing_feature_files:
        error_message = (
            f"Błąd: Brak plików cech dla: {', '.join(missing_feature_files)}. "
            f"Zaleca się uruchomienie funkcji `process_dataset` w pliku `ResNet_porównanie.ipynb`, "
            f"aby wygenerować brakujące pliki w folderze `processed_features`."
        )
        raise FileNotFoundError(error_message)
        
    # Weryfikacja, czy znaleziono jakiekolwiek działające pary model-cechy
    if not model_paths:
        raise ValueError("Brak działających modeli i odpowiadających im plików cech! "
                         "Zaleca się sprawdzenie folderów 'feature_comparison_results' i 'processed_features'.")

    # Normalizacja wag, aby sumowały się do 1
    num_features = len(model_paths)
    initial_weights = {k: 1.0 / num_features for k in model_paths.keys()}
    
    # Tworzenie konfiguracji
    config = {
        "model_paths": model_paths,
        "model_class": AudioResNet,
        "feature_files": feature_files,
        "initial_weights": initial_weights,
        "output_dir": ENSEMBLE_OUTPUT_DIR,
        "experiment_name": f"ensemble_model_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
        "test_split": TEST_SPLIT,
        "batch_size": BATCH_SIZE,
        "optimization": {
            "n_trials": OPTUNA_TRIALS,
            "timeout": OPTUNA_TIMEOUT,
            "n_folds": CV_FOLDS
        },
        "class_names": CLASS_NAMES,
        "random_seed": SEED
    }
    
    
    print(f"\nZnaleziono {len(model_paths)} działających par model-cechy:")
    for feat in model_paths.keys():
        print(f"  - {feat}")
    
    return config



In [12]:
selected_features = ["melspectrogram", "mfcc", "chroma"]

CONFIG = auto_configure_ensemble(feature_types=selected_features)

Typy cech wybrane do modelu ensemble: melspectrogram, mfcc, chroma

Znaleziono 3 działających par model-cechy:
  - melspectrogram
  - mfcc
  - chroma


In [16]:
# Komórka odpowiedzialna za optymalizację wag modelu, w której definiowane są parametry optymalizacji
n_trials = CONFIG["optimization"]["n_trials"]
timeout = CONFIG["optimization"]["timeout"]
n_folds = CONFIG["optimization"]["n_folds"]
test_size = CONFIG["test_split"]

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
# Wykorzystanie podstawowego output_dir z konfiguracji
base_output_dir = CONFIG.get("output_dir", "ensemble_outputs") 
output_dir = os.path.join(base_output_dir, f"ensemble_run_{timestamp}")

trainer = EnsembleModelTrainer(
    model_paths=CONFIG["model_paths"],
    feature_files=CONFIG["feature_files"],
    model_class=CONFIG["model_class"],
    output_dir=output_dir
)
# custom_weights = {
# "chroma": 0.13857384163072003,
# "melspectrogram": 0.31651310994496806,
# "mfcc": 0.544913048424312
# }
best_score, best_weights = trainer.optimize_weights(
    n_trials=n_trials,
    timeout=timeout,
    n_folds=n_folds,
    test_size=test_size
)

weights_to_use = best_weights
ensemble_model, test_results = trainer.train_and_evaluate(
    weights=weights_to_use,
    test_size=CONFIG["test_split"],
    batch_size=CONFIG["batch_size"]
)





# Sprawdź czy model i konfiguracja są dostępne
if 'ensemble_model' in locals() and 'CONFIG' in locals():
    print("\nModel ensemble i konfiguracja znalezione!")
    
    # Utwórz katalog wyjściowy z timestampem
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    output_dir = f"exported_models/onnx_{timestamp}"
    os.makedirs(output_dir, exist_ok=True)
    print(f"Katalog wyjściowy: {output_dir}")
    
    # Konfiguracja kwantyzacji
    quantize_model = True         # Czy kwantyzować model po eksporcie
    quantization_type = "dynamic" # Typ kwantyzacji 
    quantization_dtype = "uint8"  # Typ danych kwantyzacji
    
    print("\nRozpoczynam eksport modelu...")
    
    # Eksport modelu do formatu ONNX
    try:
        export_result = export_model_to_onnx(
            ensemble_model=ensemble_model,
            output_dir=output_dir,
            class_names=CONFIG.get("class_names"),
            export_params={
                "quantize_model": quantize_model,
                "quantization_type": quantization_type,
                "quantization_dtype": quantization_dtype,
                "use_cuda": device.type == 'cuda'
            }
        )
    except TypeError as e:
        print(f"Błąd: {e}")
        try:
            sig = inspect.signature(export_model_to_onnx)
            print(f"Oczekiwane parametry: {list(sig.parameters.keys())}")
        except:
            print("Nie udało się uzyskać parametrów funkcji.")
        raise
    
    if export_result.get("success", False):
        print("\nEksport modelu ensemble do ONNX zakończony sukcesem!")
        print(f"Model ONNX zapisany w: {export_result['onnx_path']}")
        print(f"Zoptymalizowany model ONNX zapisany w: {export_result['optimized_onnx_path']}")
        
        # Rozmiar modelu
        original_size_mb = os.path.getsize(export_result['onnx_path']) / (1024*1024)
        optimized_size_mb = os.path.getsize(export_result['optimized_onnx_path']) / (1024*1024)
        print(f"Rozmiar oryginalnego modelu: {original_size_mb:.2f} MB")
        print(f"Rozmiar zoptymalizowanego modelu: {optimized_size_mb:.2f} MB")
        print(f"Redukcja rozmiaru po optymalizacji: {export_result.get('optimization_reduction', 0):.2f}%")
        
        # Sprawdź czy kwantyzacja została wykonana w funkcji eksportu
        if "quantized_onnx_path" in export_result and os.path.exists(export_result['quantized_onnx_path']):
            quantized_size_mb = os.path.getsize(export_result['quantized_onnx_path']) / (1024*1024)
            print(f"Skwantyzowany model ONNX zapisany w: {export_result['quantized_onnx_path']}")
            print(f"Rozmiar skwantyzowanego modelu: {quantized_size_mb:.2f} MB")
            print(f"Redukcja rozmiaru po kwantyzacji: {export_result.get('quantization_reduction', 0):.2f}%")
        else:
            print("Kwantyzacja nie została przeprowadzona w ramach procesu eksportu.")
        
        # Informacje o metadanych i weryfikacji
        print(f"Metadane modelu zapisane w: {export_result.get('metadata_path', 'N/A')}")
        
        # Wyświetl wyniki weryfikacji
        if "verification_result" in export_result:
            result = export_result["verification_result"]
            if result.get("success", False):
                print("\nWeryfikacja modelu zakończona powodzeniem")
                if "max_diff" in result:
                    print(f"Maksymalna różnica między PyTorch i ONNX: {result['max_diff']:.6f}")
                if "has_warning" in result and result["has_warning"]:
                    print(f"Ostrzeżenie: {result.get('warning', 'Wykryto różnice, ale mieszczą się w tolerancji')}")
            else:
                print(f"\nWeryfikacja modelu nie powiodła się: {result.get('error', 'Nieznany błąd')}")
    else:
        print("\nEksport modelu się nie powiódł")
        print(f"Błąd: {export_result.get('error', 'Nieznany błąd')}")
else:
    print("Błąd: Nie znaleziono zmiennych ensemble_model i/lub CONFIG.")


Próba załadowania modelu dla cechy 'melspectrogram' z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/melspectrogram\best_model_melspectrogram_20250517_001727.pt
Model załadowany pomyślnie z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/melspectrogram\best_model_melspectrogram_20250517_001727.pt
Pomyślnie załadowano model dla cechy 'melspectrogram'
Próba załadowania modelu dla cechy 'mfcc' z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/mfcc\best_model_mfcc_20250517_004119.pt
Model załadowany pomyślnie z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/mfcc\best_model_mfcc_20250517_004119.pt
Pomyślnie załadowano model dla cechy 'mfcc'
Próba załadowania modelu dla cechy 'chroma' z c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\feature_comparison_results/chroma\bes

2025/05/19 09:20:51 INFO mlflow.tracking.fluent: Experiment with name 'ensemble_optimization_20250519_092050' does not exist. Creating a new experiment.
[I 2025-05-19 09:20:51,076] A new study created in memory with name: ensemble_optimization_20250519_092050
[I 2025-05-19 09:21:16,387] Trial 0 finished with value: 0.98837749883775 and parameters: {'melspectrogram': 0.3745401188473625, 'mfcc': 0.9507143064099162, 'chroma': 0.7319939418114051}. Best is trial 0 with value: 0.98837749883775.
[I 2025-05-19 09:21:40,063] Trial 1 finished with value: 0.9869827986982799 and parameters: {'melspectrogram': 0.5986584841970366, 'mfcc': 0.15601864044243652, 'chroma': 0.15599452033620265}. Best is trial 0 with value: 0.98837749883775.
[I 2025-05-19 09:22:02,756] Trial 2 finished with value: 0.9888423988842399 and parameters: {'melspectrogram': 0.05808361216819946, 'mfcc': 0.8661761457749352, 'chroma': 0.6011150117432088}. Best is trial 2 with value: 0.9888423988842399.
[I 2025-05-19 09:22:25,031] T


Model ensemble i konfiguracja znalezione!
Katalog wyjściowy: exported_models/onnx_20250519_093924

Rozpoczynam eksport modelu...
Katalog wyjściowy: exported_models/onnx_20250519_093924

Generowanie przykładowych danych wejściowych...
Wymiary wejść:
 - mel_input:   torch.Size([1, 1, 128, 130])
 - mfcc_input:  torch.Size([1, 1, 40, 130])
 - chroma_input: torch.Size([1, 1, 12, 130])

--- Eksport modelu do formatu ONNX ---
Rozpoczynam eksport modelu ensemble do formatu ONNX: exported_models/onnx_20250519_093924\ensemble_model.onnx
Test forward pass zakończony sukcesem. Kształt wyjścia: torch.Size([1, 6])
Wymiary wejść:
 - mel_input:   torch.Size([1, 1, 128, 130])
 - mfcc_input:  torch.Size([1, 1, 40, 130])
 - chroma_input: torch.Size([1, 1, 12, 130])


c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\.venv\lib\site-packages\onnxscript\converter.py:816: FutureWarning: 'onnxscript.values.Op.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\.venv\lib\site-packages\onnxscript\converter.py:816: FutureWarning: 'onnxscript.values.OnnxFunction.param_schemas' is deprecated in version 0.1 and will be removed in the future. Please use '.op_signature' instead.
  param_schemas = callee.param_schemas()
c:\Users\kubas\Desktop\Projekt dyplomowy\Audio-Emotion-Recognition\src\export_scripts\ensemble_onnx_wrapper.py:97: UserWarning: # 'dynamic_axes' is not recommended when dynamo=True, and may lead to 'torch._dynamo.exc.UserError: Constraints violated.' Supply the 'dynamic_shapes' argument instead if export is unsuccessful.
  torch.onnx.export(


[torch.onnx] Obtain model graph for `EnsembleONNXWrapper([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `EnsembleONNXWrapper([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...


...

[torch.onnx] Translate the graph into ONNX... ✅
Applied 3 of general pattern rewrite rules.
Eksport do ONNX zakończony pomyślnie!
Weryfikacja modelu ONNX...
✓ Model ONNX poprawny!
Rozmiar modelu: 0.49 MB
Liczba węzłów grafu: 226
Liczba inicjalizatorów: 366
Model wyeksportowany do: exported_models/onnx_20250519_093924\ensemble_model.onnx

--- Optymalizacja modelu ONNX ---
Rozpoczęcie optymalizacji modelu ONNX: exported_models/onnx_20250519_093924\ensemble_model.onnx
Utworzono kopię zapasową oryginalnego modelu: exported_models/onnx_20250519_093924\ensemble_model.onnx.backup
Rozmiar oryginalnego modelu: 0.49 MB
Używanie bezpiecznych optymalizacji...
Model po optymalizacji poprawny.
Ostrzeżenie: Zoptymalizowany model jest znacznie większy (128.08 MB) niż oryginalny (0.49 MB). Używanie oryginału.

--- Kwantyzacja modelu ONNX ---
Rozpoczęcie kwantyzacji modelu ONNX: exported_models/onnx_20250519_093924\ensemble_model_optimized.onnx
Model ONNX poprawny przed kwantyzacją.
Shape inference prze

Użycie kwantyzacji do unsigned int8
Próba kwantyzacji podstawowej...


Podstawowa kwantyzacja nie powiodła się: [ShapeInferenceError] Inferred shape and existing shape differ in dimension 0: (512) vs (6)
Próba kwantyzacji z pominięciem problematycznych operatorów...
Pominięcie operatorów: ['Reshape', 'Transpose', 'Concat', 'Slice', 'Squeeze', 'Unsqueeze']
Również nie powiodła się kwantyzacja z pominięciem operatorów: quantize_dynamic() got an unexpected keyword argument 'op_types_to_exclude'
Próba ostatecznej, minimalnej kwantyzacji...
Kwantyzacja tylko operatorów: ['Conv', 'MatMul']
Wszystkie próby kwantyzacji nieudane: [ShapeInferenceError] Inferred shape and existing shape differ in dimension 0: (512) vs (6)
Ostrzeżenie: Kwantyzacja modelu nie powiodła się: Wszystkie próby kwantyzacji nieudane: [ShapeInferenceError] Inferred shape and existing shape differ in dimension 0: (512) vs (6)
Kontynuowanie bez kwantyzacji.

--- Weryfikacja modelu ONNX ---
Dostępni dostawcy ONNX Runtime: ['AzureExecutionProvider', 'CPUExecutionProvider']
Używam CPU do inferencj

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>